<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/Neo4jOpenAIApoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.9.0-py3-none-any.whl size=259467 sha256=57e9cad008323c6a8975757f51e5db0a43a093542857eb0293800bb3a7e52a3f
  Stored in directory: /root/.cache/pip/wheels/aa/d6/e2/3534952aaddb39337f01f3fd66b3f3f2dd65051306a614af92
Successfully built neo4j


In [2]:

# Define Neo4j connections
from neo4j import GraphDatabase
host = 'bolt://44.215.124.63:7687'
user = 'neo4j'
password = 'steel-orders-reproduction'
driver = GraphDatabase.driver(host,auth=(user, password))

def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return result.to_df()

In [3]:
run_query("""
MATCH (m:Movie)
RETURN m.title AS title
LIMIT 3
""")

,title
0,The Matrix
1,The Matrix Reloaded
2,The Matrix Revolutions


In [8]:
print(run_query("""
MATCH (m:Movie)
MATCH (m)-[r:ACTED_IN|DIRECTED]-(t)
WITH m, type(r) as type, collect(t.name) as names
WITH m, type+": "+reduce(s="", n IN names | s + n + ", ") as types
WITH m, collect(types) as contexts
WITH m, "Movie title: "+ m.title + " year: "+coalesce(m.released,"") +" plot: "+ coalesce(m.tagline,"")+"\n" +
       reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
RETURN context LIMIT 1
""")['context'][0])

Movie title: The Matrix year: 1999 plot: Welcome to the Real World
ACTED_IN: Emil Eifrem, Hugo Weaving, Laurence Fishburne, Carrie-Anne Moss, Keanu Reeves
DIRECTED: Lana Wachowski, Lilly Wachowski



In [39]:
openai_api_key = "OPENAI_API_KEY"

In [42]:
run_query("""
CALL apoc.periodic.iterate(
  'MATCH (m:Movie) RETURN id(m) AS id',
  'MATCH (m:Movie)
   WHERE id(m) = id
   MATCH (m)-[r:ACTED_IN|DIRECTED]-(t)
   WITH m, type(r) as type, collect(t.name) as names
   WITH m, type+": "+reduce(s="", n IN names | s + n + ", ") as types
   WITH m, collect(types) as contexts
   WITH m, "Movie title: "+ m.title + " year: "+coalesce(m.released,"") +" plot: "+ coalesce(m.tagline,"")+"\n" +
        reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
   CALL apoc.ml.openai.embedding([context], $apiKey) YIELD embedding
   SET m.embedding = embedding',
  {batchSize:1, retries:3, params: {apiKey: $apiKey}})
""", {'apiKey': openai_api_key})['errorMessages'][0]

{}

In [43]:
system_prompt = """
You are an assistant that helps to generate text to form nice and human understandable answers based.
The latest prompt contains the information, and you need to generate a human readable response based on the given information.
Make the answer sound as a response to the question. Do not mention that you based the result on the given information.
Do not add any additional information that is not explicitly provided in the latest prompt.
I repeat, do not add any information that is not explicitly given.
"""

def generate_user_prompt(question, context):
   return f"""
   The question is {question}
   Answer the question by using the provided information:
   {context}
   """

def retrieve_context(question, k=3):
  data = run_query("""
    // retrieve the embedding of the question
    CALL apoc.ml.openai.embedding([$question], $apiKey) YIELD embedding
    // match relevant movies
    MATCH (m:Movie)
    WITH m, gds.similarity.cosine(embedding, m.embedding) AS score
    ORDER BY score DESC
    LIMIT toInteger($k)
    MATCH (m)-[r:ACTED_IN|DIRECTED]-(t)
    WITH m, type(r) as type, collect(t.name) as names
    WITH m, type+": "+reduce(s="", n IN names | s + n + ", ") as types
    WITH m, collect(types) as contexts
    WITH m, "Movie title: "+ m.title + " year: "+coalesce(m.released,"") +" plot: "+ coalesce(m.tagline,"")+"\n" +
          reduce(s="", c in contexts | s + substring(c, 0, size(c)-2) +"\n") as context
    RETURN context
  """, {'question': question, 'k': k, 'apiKey': openai_api_key})
  return data['context'].to_list()

def generate_answer(question):
  # Retrieve context
  context = retrieve_context(question)
  # Print context
  for c in context:
    print(c)
  # Generate answer
  response = run_query("""
  CALL apoc.ml.openai.chat([{role:'system', content: $system},
                      {role: 'user', content: $user}], $apiKey) YIELD value
  RETURN value.choices[0].message.content AS answer
  """, {'system': system_prompt, 'user': generate_user_prompt(question, context), 'apiKey': openai_api_key})
  return response['answer'][0]



In [44]:
generate_answer("Who played in the Matrix?")

Movie title: The Matrix year: 1999 plot: Welcome to the Real World
ACTED_IN: Emil Eifrem, Hugo Weaving, Laurence Fishburne, Carrie-Anne Moss, Keanu Reeves
DIRECTED: Lana Wachowski, Lilly Wachowski

Movie title: The Matrix Reloaded year: 2003 plot: Free your mind
DIRECTED: Lana Wachowski, Lilly Wachowski
ACTED_IN: Hugo Weaving, Laurence Fishburne, Carrie-Anne Moss, Keanu Reeves

Movie title: The Matrix Revolutions year: 2003 plot: Everything that has a beginning has an end
DIRECTED: Lana Wachowski, Lilly Wachowski
ACTED_IN: Hugo Weaving, Laurence Fishburne, Carrie-Anne Moss, Keanu Reeves



"Keanu Reeves, Laurence Fishburne, Carrie-Anne Moss, Hugo Weaving, and Emil Eifrem played in the movie 'The Matrix' which was released in 1999. They also starred in the sequels - 'The Matrix Reloaded' and 'The Matrix Revolutions', both released in 2003. All three movies were directed by Lana Wachowski and Lilly Wachowski."

In [45]:
generate_answer("Which movie was released in 2000?")

Movie title: Cast Away year: 2000 plot: At the edge of the world, his journey begins.
ACTED_IN: Helen Hunt, Tom Hanks
DIRECTED: Robert Zemeckis

Movie title: Jerry Maguire year: 2000 plot: The rest of his life begins now.
ACTED_IN: Jerry O'Connell, Bonnie Hunt, Jay Mohr, Cuba Gooding Jr., Jonathan Lipnicki, Renee Zellweger, Kelly Preston, Regina King, Tom Cruise
DIRECTED: Cameron Crowe

Movie title: The Matrix Reloaded year: 2003 plot: Free your mind
DIRECTED: Lana Wachowski, Lilly Wachowski
ACTED_IN: Hugo Weaving, Laurence Fishburne, Carrie-Anne Moss, Keanu Reeves



"The movie that was released in 2000 is Cast Away. It's a drama film about a man named Chuck Nolan who becomes stranded on a deserted island after a plane crash. The movie stars Tom Hanks and Helen Hunt, and was directed by Robert Zemeckis."

In [46]:
generate_answer("Recommend me a comedy with Jack Nicholson?")

Movie title: As Good as It Gets year: 1997 plot: A comedy from the heart that goes for the throat.
ACTED_IN: Helen Hunt, Jack Nicholson, Cuba Gooding Jr., Greg Kinnear
DIRECTED: James L. Brooks

Movie title: One Flew Over the Cuckoo's Nest year: 1975 plot: If he's crazy, what does that make you?
ACTED_IN: Danny DeVito, Jack Nicholson
DIRECTED: Milos Forman

Movie title: Something's Gotta Give year: 2003 plot: 
ACTED_IN: Keanu Reeves, Diane Keaton, Jack Nicholson
DIRECTED: Nancy Meyers



'I would highly recommend the comedy movie "As Good as It Gets" starring Jack Nicholson along with other great stars like Helen Hunt, Cuba Gooding Jr., and Greg Kinnear. Directed by James L. Brooks, the movie from the heart that goes for the throat was released in 1997 and is sure to leave you laughing out loud. Another option you can consider is "Something\'s Gotta Give" that released in 2003, directed by Nancy Meyers and also featuring Jack Nicholson along with Keanu Reeves and Diane Keaton.'